## 01_backtesting-py_simple.ipynb
- **Goal:** Evaluate a long-only SMA crossover strategy on BTCUSDT hourly data with the `backtesting.py` framework.
- **Data preparation:** Retrieves candles through `src.data.data_pipeline.get_historical_data`, then converts the Polars DataFrame to pandas for compatibility with `Backtest`.
- **Strategy logic:** Implements `SmaCross_bt`, which tracks short/long simple moving averages, buys on upward crossovers, and exits on downward crossovers.
- **Backtest run:** Executes `Backtest` with $1M starting cash, 0.2% commission, and exclusive orders; prints the resulting performance statistics and generates the HTML equity plot.
- **Optimization experiments:** Runs a 300-trial Optuna study optimizing both return and Sharpe ratio (multi-objective) followed by a single-objective variant that combines the same metrics, then re-runs the backtest with the chosen parameters (`n_short=47`, `n_long=67`).

In [1]:
import os
import sys
# Add src folder to Python path
root_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(root_path)

## Load data

In [2]:
from src.data.data_pipeline import get_historical_data
from src.utils.utils import polars_to_pandas

In [3]:
df = get_historical_data(download=False)
df.head()

open_time,open,high,low,close,volume,close_time,quote_asset_volume,trades,taker_base_vol,taker_quote_vol,ignore
datetime[ms],f64,f64,f64,f64,f64,datetime[ms],f64,i64,f64,f64,str
2024-09-24 16:00:00,63159.99,63372.45,63088.02,63262.0,524.87788,2024-09-24 16:59:59.999,3.3183e7,175340,250.56544,1.5842e7,"""0"""
2024-09-24 17:00:00,63262.01,64000.0,63241.75,63775.86,1504.89392,2024-09-24 17:59:59.999,9.5883e7,264552,766.68339,4.8839e7,"""0"""
2024-09-24 18:00:00,63775.54,63886.0,63540.48,63727.58,804.94922,2024-09-24 18:59:59.999,5.1308e7,178871,316.93669,2.0201e7,"""0"""
2024-09-24 19:00:00,63727.58,64375.0,63682.93,64292.34,1316.46046,2024-09-24 19:59:59.999,8.4309e7,206037,831.68881,5.3270e7,"""0"""
2024-09-24 20:00:00,64292.35,64596.53,64093.15,64215.01,1610.30279,2024-09-24 20:59:59.999,1.0354e8,254197,817.49704,5.2583e7,"""0"""


## Defining strategy

In [4]:
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting.test import SMA
from backtesting import Backtest

class SmaCross_bt(Strategy):
    """
    A simple SMA Cross strategy:
    - If short SMA crosses above long SMA, enter a long position.
    - If short SMA crosses below long SMA, close the position.
    """

    n_short = 30
    n_long = 100

    def init(self):
        # Convert the series to indicators used by backtesting.py
        price = self.data.Close
        self.sma_short = self.I(SMA, price, self.n_short)
        self.sma_long = self.I(SMA, price, self.n_long)

    def next(self):
        # If short SMA crosses above long SMA, and not already in a trade:
        if crossover(self.sma_short, self.sma_long):
            self.buy()

        # If short SMA crosses below long SMA, close any open position:
        elif crossover(self.sma_long, self.sma_short):
            self.position.close()

/Users/ezequiel.rivero/personal/repos/Crypto-Backtester-Duel/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/ezequiel.rivero/personal/repos/Crypto-Backtester-Duel/.venv/lib/python3.12/site-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [5]:
def run_bt_sma_backtest(df_pl) -> None:
    df_pd = polars_to_pandas(df_pl)
    bt = Backtest(df_pd, SmaCross_bt, cash=1_000_000, commission=0.002, exclusive_orders=True)
    stats = bt.run()
    return stats, bt

stats, bt = run_bt_sma_backtest(df)
print(stats)
bt.plot()

Start                     2024-09-24 16:00:00
End                       2025-09-24 15:00:00
Duration                    364 days 23:00:00
Exposure Time [%]                    55.73059
Equity Final [$]                1094161.98954
Equity Peak [$]                 1535359.05996
Commissions [$]                  244949.18046
Return [%]                             9.4162
Buy & Hold Return [%]                72.45817
Return (Ann.) [%]                      9.3893
Volatility (Ann.) [%]                36.43405
CAGR [%]                              9.41732
Sharpe Ratio                          0.25771
Sortino Ratio                         0.46411
Calmar Ratio                          0.26886
Alpha [%]                           -27.91721
Beta                                  0.51524
Max. Drawdown [%]                   -34.92262
Avg. Drawdown [%]                    -2.13122
Max. Drawdown Duration      281 days 01:00:00
Avg. Drawdown Duration        7 days 05:00:00
# Trades                          

GridPlot(id='p1356', ...)

In [6]:
import optuna
from backtesting import Backtest

# Objective function for Optuna
def multi_objective(trial):
    # 1) Suggest parameters
    n_short = trial.suggest_int('n_short', 5, 50)
    # ensure n_long > n_short
    n_long  = trial.suggest_int('n_long', n_short + 1, 200)

    # 2) Update strategy class vars
    SmaCross_bt.n_short = n_short
    SmaCross_bt.n_long  = n_long

    # 3) Run backtest
    stats, _ = run_bt_sma_backtest(df)

    # 4) Extract objectives
    ret    = stats['Return [%]']    # e.g. 12.34
    sharpe = stats['Sharpe Ratio']  # e.g. 1.23

    # Optuna multi-objective: return a tuple
    return ret, sharpe

# Create a multi-objective study: maximize both return and sharpe
study = optuna.create_study(
    directions=['maximize', 'maximize'],
    study_name='sma_cross_multiobj'
)

# Run optimization
study.optimize(multi_objective, n_trials=300)
print("Number of Pareto-optimal trials:", len(study.best_trials))
for t in study.best_trials:
    print(f"  trial#{t.number}: Return={t.values[0]:.2f}%, Sharpe={t.values[1]:.2f}, "
          f"n_short={t.params['n_short']}, n_long={t.params['n_long']}")

[I 2025-09-24 19:49:59,630] A new study created in memory with name: sma_cross_multiobj
[I 2025-09-24 19:49:59,675] Trial 0 finished with values: [5.414460287999897, 0.15459937730819806] and parameters: {'n_short': 43, 'n_long': 94}.
[I 2025-09-24 19:49:59,715] Trial 1 finished with values: [2.5033715439999478, 0.07530276955704009] and parameters: {'n_short': 39, 'n_long': 162}.
[I 2025-09-24 19:49:59,755] Trial 2 finished with values: [8.816944222000032, 0.2565520615658232] and parameters: {'n_short': 36, 'n_long': 180}.
[I 2025-09-24 19:49:59,792] Trial 3 finished with values: [1.7803458280000604, 0.05545077934055786] and parameters: {'n_short': 37, 'n_long': 139}.
[I 2025-09-24 19:49:59,830] Trial 4 finished with values: [12.456652460000035, 0.3361629158411672] and parameters: {'n_short': 31, 'n_long': 60}.
/var/folders/69/yp90qml119q4774f6bcf_0ym0000gp/T/ipykernel_32328/2067561108.py:4: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=

Number of Pareto-optimal trials: 1
  trial#122: Return=32.81%, Sharpe=0.72, n_short=49, n_long=73


In [7]:
import optuna
from backtesting import Backtest

# Objective function for Optuna
def single_objective(trial):
    # 1) Suggest parameters
    n_short = trial.suggest_int('n_short', 5, 50)
    # ensure n_long > n_short
    n_long  = trial.suggest_int('n_long', n_short + 1, 200)

    # 2) Update strategy class vars
    SmaCross_bt.n_short = n_short
    SmaCross_bt.n_long  = n_long

    # 3) Run backtest
    stats, _ = run_bt_sma_backtest(df)

    # 4) Extract objectives
    ret    = stats['Return [%]']    # e.g. 12.34
    sharpe = stats['Sharpe Ratio']  # e.g. 1.23

    return ret + 10*sharpe

# Create a multi-objective study: maximize both return and sharpe
study = optuna.create_study(
    direction='maximize',
    study_name='sma_cross_singleobj'
)

# Run optimization
study.optimize(single_objective, n_trials=300)
print("Number of Pareto-optimal trials:", len(study.best_trials))
for t in study.best_trials:
    print(f"  trial#{t.number}: "
          f"n_short={t.params['n_short']}, n_long={t.params['n_long']}")

[I 2025-09-24 19:50:49,096] A new study created in memory with name: sma_cross_singleobj
[I 2025-09-24 19:50:49,151] Trial 0 finished with value: -1.5890566701851818 and parameters: {'n_short': 18, 'n_long': 142}. Best is trial 0 with value: -1.5890566701851818.
[I 2025-09-24 19:50:49,195] Trial 1 finished with value: 17.87565097850241 and parameters: {'n_short': 32, 'n_long': 129}. Best is trial 1 with value: 17.87565097850241.
[I 2025-09-24 19:50:49,237] Trial 2 finished with value: -1.5101254948302336 and parameters: {'n_short': 47, 'n_long': 133}. Best is trial 1 with value: 17.87565097850241.
[I 2025-09-24 19:50:49,274] Trial 3 finished with value: 11.284893657316953 and parameters: {'n_short': 37, 'n_long': 165}. Best is trial 1 with value: 17.87565097850241.
[I 2025-09-24 19:50:49,312] Trial 4 finished with value: 9.614748028830125 and parameters: {'n_short': 50, 'n_long': 84}. Best is trial 1 with value: 17.87565097850241.
[I 2025-09-24 19:50:49,351] Trial 5 finished with value

Number of Pareto-optimal trials: 1
  trial#274: n_short=43, n_long=70


In [8]:
SmaCross_bt.n_short = 47
SmaCross_bt.n_long  = 67
stats, bt = run_bt_sma_backtest(df)
print(stats)
bt.plot()

Start                     2024-09-24 16:00:00
End                       2025-09-24 15:00:00
Duration                    364 days 23:00:00
Exposure Time [%]                    54.58904
Equity Final [$]                1266902.54922
Equity Peak [$]                 1639846.53594
Commissions [$]                  419503.44078
Return [%]                           26.69025
Buy & Hold Return [%]                73.07435
Return (Ann.) [%]                    26.60839
Volatility (Ann.) [%]                43.08171
CAGR [%]                             26.69368
Sharpe Ratio                          0.61763
Sortino Ratio                          1.3588
Calmar Ratio                          0.94655
Alpha [%]                           -11.75714
Beta                                  0.52614
Max. Drawdown [%]                   -28.11102
Avg. Drawdown [%]                     -1.7937
Max. Drawdown Duration      281 days 01:00:00
Avg. Drawdown Duration        6 days 05:00:00
# Trades                          

GridPlot(id='p1736', ...)